In [ ]:
### This script is to integrate DMIR / DeepSNIF analysis from the IMC_denoise package into the steinbock (https://bodenmillergroup.github.io/steinbock/) workflow
### Step of steinbock pipeline: after the conversion of mcd into .tiff files 

### Edited from the example DeepSNiF train/run jupyter notebook script in IMC_denoise
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tp
from IMC_Denoise.IMC_Denoise_main.DIMR import DIMR
from IMC_Denoise.IMC_Denoise_main.DeepSNiF import DeepSNiF
from IMC_Denoise.DeepSNiF_utils.DeepSNiF_DataGenerator import DeepSNiF_DataGenerator

if 'generated_patches' in globals():
    del generated_patches

train_directory = "C:\\Users\\......\\training_img" # change this to the directory of your training images 
Raw_directory = "C:\\Users\\......\\img" # change this directory to your directory of pre-denoised tiffs (always ends with \\img for steinbock exports).
output_directory = "C:\\Users\\......\\output" # change this directory to where you want the denoised images to be written to

# Make all three directories the same to train on the whole dataset, then overwrite your current files with the denoised versions

# The most convenient way to integrate steinbock and IMC_denoise is to have the output directly overwrite the img folder.
# This code chunk creates a pre_denoised copy of the img folder if you plan to use the overwrite.
# This means if you need to undo/redo the denoising, steinbock will not need to convert the mcd's to tiffs again
if Raw_directory == output_directory:    
    if os.path.isdir(Raw_directory + "_pre_denoise") == False:
        shutil.copytree(Raw_directory,(Raw_directory + "_pre_denoise"))
    else:
        print('img_pre_denoise folder already exists, will not copy current img directory')
        
        
# Choose what channels you want to denoise:
channel_names = [17,18]     # list of integers corresponding to the channels of interest / channels you want to denoise
                            # IMPORTANT! only run channels that you want to denoise with DIMR and DeepSNiF -- examine images first before running
                            # if overwriting (raw_directory == output_directory), you cannot directly re-run the same channel.

In [2]:
### Step 0.1: edit IMC_denoise so that it will ingest multi-channle tiffs directly from the directory, instead of requiring the directory structure of the original IMC_denoise package
# Edits in this file: ...\IMC_Denoise\DeepSNIF_utils\DeepSniF_DataGenerator.py
display()

In [ ]:
## Step 0.2: set up a function that integrates all the training functions of IMC_denoise
# Just done here so that the code for iterating through the channels is simpler
def DeepSNIF_train(channel_name, n_neighbours = 4, n_iter = 3, window_size = 3, train_epoches = 25, train_initial_lr = 1e-3, 
                  train_batch_size = 128, pixel_mask_percent = 0.2, val_set_percent = 0.15, loss_function = "I_divergence",
                  weights_name = None, loss_name = None, weights_save_directory = None, lambda_HF = 3e-6):
    '''
    Docstring?
    '''
    DataGenerator = DeepSNiF_DataGenerator(run_type = 'multi_channel_tiff', channel_name = channel_name, ratio_thresh = 0.8,
                                           patch_row_size = 64, patch_col_size = 64, row_step = 60, col_step = 60,
                                           n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size)
    generated_patches = DataGenerator.generate_patches_from_directory(load_directory = train_directory)
    print('The shape of the generated training set for channel ' + str(channel_name)  + ' is ' + str(generated_patches.shape) + '.')
    is_load_weights = False # Use the trained model directly. Will not read from saved one.
    deepsnif = DeepSNiF(train_epoches = train_epoches, 
                    train_learning_rate = train_initial_lr,
                    train_batch_size = train_batch_size,
                    mask_perc_pix = pixel_mask_percent,
                    val_perc = val_set_percent,
                    loss_func = loss_function,
                    weights_name = weights_name,
                    loss_name = loss_name,
                    weights_dir = weights_save_directory, 
                    is_load_weights = is_load_weights,
                    lambda_HF = lambda_HF)
    train_loss, val_loss = deepsnif.train(generated_patches)
    return(n_neighbours, n_iter, window_size, train_loss, val_loss, deepsnif)


In [ ]:
# Step 1: iterate through the channels, training  and then running for each image
if len(channel_names) < 1:  ## if no channel names are specified, all channels are run! Depending on how many channels / how long the training -- that would take a very long time
    print('No channels specified! Will denoise all channels in provided .tiffs')
    tif = TiffFile(Raw_directory + "\\" + listdir(Raw_directory)[0])  # reads the first image in the dataset to get the number of channels in the dataset (assumes a consistent number)
    channel_number = len(tif.pages)
    i = 0
    for j in np.range(0,channel_number):
        channel_names.append(i)
        i += 1
for i in channel_names:
    n_neighbours, n_iter, window_size, train_loss, val_loss, deepsnif = DeepSNIF_train(i)
    for img in os.listdir(Raw_directory):
        Img_raw = tp.TiffFile(Raw_directory + "\\" + img).pages[i].asarray()
        Img_DIMR_DeepSNiF = deepsnif.perform_IMC_Denoise(Img_raw, n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size)
        numpy_tiff = tp.imread(Raw_directory + "\\" + img)
        numpy_tiff[i] = Img_DIMR_DeepSNiF
        tp.imwrite((output_directory + "\\" + img),numpy_tiff, photometric='minisblack')

# Steinbock should now be able to seemlessly work with the denoised files